Forget Gate: This gate decide which information is important and should be stored and which information to forget. It removes the non important information from neuron cell. 
This gate takes 2 input- one is the output generated by previous cell and other is input of current cell. Following required bias and weights are added and multiplied and sigmoid function is applied to the value. A value between 0 and 1 is generated and based on this we decide which information to keep. If value is 0 the forget gate will remove that information and if value is 1 then information is important and has to be remembered.

Input Gate: This gate is used to add information to neuron cell. It is responsible of what values should be added to cell by using activation function like sigmoid. It creates an array of information that has to be added. This is done by using another activation function called tanh. It generates a values between -1 and 1. The sigmoid function act as a filter and regulate what information has to be added in cell.

Output Gate: This gate is responsible for selecting important information from current cell and show it as output. It creates a vector of values using tanh function which ranges from -1 to 1. It uses previous output and current input as a regulator which also includes sigmoid function and decides which values should be shown as output.

In [1]:
#All Libraries required 
import numpy
import sys
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [3]:
file = open("Two-tail.txt").read()

In [ ]:
# Text cannot be processed so we have to convert it to readable form.
# This can be done by 2 methods a-one hot encoding b-word embedding.
# 1.one hot encoding involves converting text into 1's and 0's vector. It will create 
#   bag of words which represent the freq of each word in document.
# 2.Word embeddings represents text words a vector of real numbers. It can use more numbers than 0 and 1

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
from nltk.tokenize import sent_tokenize, word_tokenize
sentence=(file)
sentence=sentence.replace('\n',"")
sentence=(sent_tokenize(sentence))
print(sentence)

['I.', 'The PeriodIt was the best of times,it was the worst of times,it was the age of wisdom,it was the age of foolishness,it was the epoch of belief,it was the epoch of incredulity,it was the season of Light,it was the season of Darkness,it was the spring of hope,it was the winter of despair,we had everything before us,we had nothing before us,we were all going direct to Heaven,we were all going direct the other way--in short, the period was so far like the present period, that some ofits noisiest authorities insisted on its being received, for good or forevil, in the superlative degree of comparison only.There were a king with a large jaw and a queen with a plain face, on thethrone of England; there were a king with a large jaw and a queen witha fair face, on the throne of France.', 'In both countries it was clearerthan crystal to the lords of the State preserves of loaves and fishes,that things in general were settled for ever.It was the year of Our Lord one thousand seven hundred 

In [6]:
import string
def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii",'ignore')
    return txt

In [7]:
corpus = [clean_text(x) for x in sentence]
corpus[:10]

['i',
 'the periodit was the best of timesit was the worst of timesit was the age of wisdomit was the age of foolishnessit was the epoch of beliefit was the epoch of incredulityit was the season of lightit was the season of darknessit was the spring of hopeit was the winter of despairwe had everything before uswe had nothing before uswe were all going direct to heavenwe were all going direct the other wayin short the period was so far like the present period that some ofits noisiest authorities insisted on its being received for good or forevil in the superlative degree of comparison onlythere were a king with a large jaw and a queen with a plain face on thethrone of england there were a king with a large jaw and a queen witha fair face on the throne of france',
 'in both countries it was clearerthan crystal to the lords of the state preserves of loaves and fishesthat things in general were settled for everit was the year of our lord one thousand seven hundred and seventyfivespiritual 

In [8]:
import nltk
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential


In [9]:
# Generating n gram sequence for Training
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

In [10]:
inp_sequences, total_words = get_sequence_of_tokens(corpus)
print(*inp_sequences[:10], sep="\n")

[1, 279]
[1, 279, 7]
[1, 279, 7, 1]
[1, 279, 7, 1, 156]
[1, 279, 7, 1, 156, 2]
[1, 279, 7, 1, 156, 2, 157]
[1, 279, 7, 1, 156, 2, 157, 7]
[1, 279, 7, 1, 156, 2, 157, 7, 1]
[1, 279, 7, 1, 156, 2, 157, 7, 1, 280]
[1, 279, 7, 1, 156, 2, 157, 7, 1, 280, 2]


In [11]:
from tensorflow import keras
import tensorflow as tf
import keras.utils as ku 
from keras import utils as np_utils
from keras.utils.np_utils import to_categorical
#from tensorflow.keras.utils import to_categorical

In [12]:
# Padding the sequence
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

In [13]:
import numpy as np
predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

In [14]:
# Model Creation
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    # ----------Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    # ----------Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    # ----------Add Output Layer
    model.add(Dense(total_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

In [15]:
model = create_model(max_sequence_len, total_words)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 201, 10)           11190     
                                                                 
 lstm (LSTM)                 (None, 100)               44400     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 1119)              113019    
                                                                 
Total params: 168,609
Trainable params: 168,609
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Training the Model
model.fit(predictors, label, epochs=95, verbose=95)

Epoch 1/95
Epoch 2/95
Epoch 3/95
Epoch 4/95
Epoch 5/95
Epoch 6/95
Epoch 7/95
Epoch 8/95
Epoch 9/95
Epoch 10/95
Epoch 11/95
Epoch 12/95
Epoch 13/95
Epoch 14/95
Epoch 15/95
Epoch 16/95
Epoch 17/95
Epoch 18/95
Epoch 19/95
Epoch 20/95
Epoch 21/95
Epoch 22/95
Epoch 23/95
Epoch 24/95
Epoch 25/95
Epoch 26/95
Epoch 27/95
Epoch 28/95
Epoch 29/95
Epoch 30/95
Epoch 31/95
Epoch 32/95
Epoch 33/95
Epoch 34/95
Epoch 35/95
Epoch 36/95
Epoch 37/95
Epoch 38/95
Epoch 39/95
Epoch 40/95
Epoch 41/95
Epoch 42/95
Epoch 43/95
Epoch 44/95
Epoch 45/95
Epoch 46/95
Epoch 47/95
Epoch 48/95
Epoch 49/95
Epoch 50/95
Epoch 51/95
Epoch 52/95
Epoch 53/95
Epoch 54/95
Epoch 55/95
Epoch 56/95
Epoch 57/95
Epoch 58/95
Epoch 59/95
Epoch 60/95
Epoch 61/95
Epoch 62/95
Epoch 63/95
Epoch 64/95
Epoch 65/95
Epoch 66/95
Epoch 67/95
Epoch 68/95
Epoch 69/95
Epoch 70/95
Epoch 71/95
Epoch 72/95
Epoch 73/95
Epoch 74/95
Epoch 75/95
Epoch 76/95
Epoch 77/95
Epoch 78/95
Epoch 79/95
Epoch 80/95
Epoch 81/95
Epoch 82/95
Epoch 83/95
Epoch 84/95
E

In [17]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list],              maxlen=max_sequence_len-1, padding='pre')
        #predicted = model.predict_classes(token_list, verbose=0)
        predicted = np.argmax(model.predict(token_list),axis=1)       


        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [18]:
# set seeds for reproducability
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

In [19]:
print (generate_text("mrs southcott had recently attained", 3, model, max_sequence_len))
print (generate_text("even the cocklaneghost", 3, model, max_sequence_len))
print (generate_text("mere messages in theearthly", 4, model, max_sequence_len))


Mrs Southcott Had Recently Attained Her Fiveandtwentiethblessed Birthday
Even The Cocklaneghost Had Been Laid
Mere Messages In Theearthly Order Of The Name
